LOADING DATA

In [1]:
import pandas as pd

In [2]:
atm_cons=pd.read_csv('../data/raw/daily_atms_consommations.csv', on_bad_lines='skip', sep=';',quotechar='"')
region_city=pd.read_csv('../data/raw/region_city.csv',sep=',')
holidays = pd.read_csv('../data/raw/fériés_date.csv', on_bad_lines='skip', sep=';', quotechar='"', encoding='ISO-8859-1')
atm_city=pd.DataFrame(pd.read_excel('../data/raw/atms_city.xls'))


C:\Users\Hp\AppData\Local\Temp\ipykernel_13664\4128178266.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  atm_cons=pd.read_csv('../data/raw/daily_atms_consommations.csv', on_bad_lines='skip', sep=';',quotechar='"')


WARNING *** file size (1218634) not 512 + multiple of sector size (512)


CLEANING ATM CONSOMMATION DATA


In [3]:
atm_cons.head()

,TERMINAL,DT,AMOUNT
0,10000,02/01/2009,17200
1,20000,02/01/2009,163300
2,30000,02/01/2009,306900
3,40000,02/01/2009,231400
4,50000,02/01/2009,241000


In [4]:
atm_cons['DT'].isna().sum()

np.int64(0)

In [5]:
atm_cons['DT'] = pd.to_datetime(atm_cons['DT'], dayfirst=True, errors='coerce')

In [6]:
terminal_dates = atm_cons.groupby('TERMINAL')['DT'].agg(['min', 'max']).reset_index()
terminal_dates

,TERMINAL,min,max
0,10000,2009-01-02,2025-01-21
1,20000,2009-01-02,2025-01-21
2,30000,2009-01-02,2016-04-18
3,40000,2009-01-02,2025-01-21
4,50000,2009-01-02,2020-01-16
...,...,...,...
2273,25550000,2025-01-02,2025-01-21
2274,25560000,2025-01-10,2025-01-21
2275,25570000,2025-01-08,2025-01-21
2276,25580000,2025-01-07,2025-01-21


In [7]:
terminal_dates['duration'] = (terminal_dates['max'] - terminal_dates['min']).dt.days
filtered_terminals = terminal_dates[terminal_dates['duration'] > 30]

In [8]:
filtered_terminal_list = filtered_terminals['TERMINAL'].unique()
len(filtered_terminal_list)

2232

In [9]:
filtered_terminal_list = filtered_terminals['TERMINAL'].unique()
atm_cons = atm_cons[atm_cons['TERMINAL'].isin(filtered_terminal_list)]

In [10]:
atm_cons = atm_cons.drop_duplicates(subset=['TERMINAL','DT'])


CLEANING ATM CITY

In [11]:
atm_city.head()

,TERMINAL_ATM_NUMBER,TERMINAL_ATM_GROUPING,GROUP_LABEL,OUTLET_NUMBER,BRANCH_NAME,CITY_CODE,CITY_NAME,ADDRESS_1,ADDRESS_2,ADDRESS_3,ADDRESS_4,ATM_ACRONYM
0,4740000,81,081 B.P. RABAT-KENITRA,8196,BENI HSSEN,349,SIDI SLIMANE,LOTISSEMENT HAY ESSALAM,SIDI SLIMANE,NaN,NaN,BENI HSSEN
1,4770000,48,048 BP FES-MEKNES,4818,FAR,480,MEKNES,ANGLE AVENUE DES FAR ET RUE,VICTOR HUGO MEKNES EL MENZEH,NaN,NaN,FAR
2,4800000,27,027 BP FES-MEKNES,2744,AIN CHKAF,270,FES,408 LOTISSEMENT NOUZHA,ROUTE AIN CHKAF FES,NaN,NaN,AIN CHKAF
3,4810000,50,050 BP NADOR-AL HOCEIMA,5014,EL KHATTABI,500,NADOR,QUARTIER EL KINDI,,NaN,NaN,EL KHATTABI
4,4780000,17,017 BCP RESEAU,1768,KHEMIS ZEMAMRA,186,KHEMIS ZEMAMRA,RTE. TNINE GHARBIA,KHEMIS ZEMAMRA,NaN,NaN,KHEMIS ZEMAMRA


In [12]:
atm_city.shape

(2452, 12)

In [13]:
atm_city.isna().sum()

TERMINAL_ATM_NUMBER         0
TERMINAL_ATM_GROUPING       0
GROUP_LABEL                 0
OUTLET_NUMBER               0
BRANCH_NAME                 0
CITY_CODE                   0
CITY_NAME                   0
ADDRESS_1                   0
ADDRESS_2                   0
ADDRESS_3                2452
ADDRESS_4                2452
ATM_ACRONYM                 0
dtype: int64

In [14]:
atm_city=atm_city.drop(columns=['ADDRESS_3','ADDRESS_4','ADDRESS_1','ADDRESS_2','ATM_ACRONYM','GROUP_LABEL','OUTLET_NUMBER','TERMINAL_ATM_GROUPING'])

In [15]:
atm_city=atm_city.rename(columns={'TERMINAL_ATM_NUMBER': 'TERMINAL'})

In [16]:
atm_city['BRANCH_NAME']=atm_city['BRANCH_NAME'].str.lower()
atm_city['CITY_NAME']=atm_city['CITY_NAME'].str.lower()


In [17]:
atm_city.drop_duplicates()

,TERMINAL,BRANCH_NAME,CITY_CODE,CITY_NAME
0,4740000,beni hssen,349,sidi slimane
1,4770000,far,480,meknes
2,4800000,ain chkaf,270,fes
3,4810000,el khattabi,500,nador
4,4780000,khemis zemamra,186,khemis zemamra
...,...,...,...,...
2447,25560000,nakhil,450,marrakech
2448,25570000,nakhil,450,marrakech
2449,25580000,al akouass,810,rabat
2450,25590000,centre d'affaire hamriya,480,meknes


CLEANING REGION/CITY

In [18]:
region_city.head()

,city,city_ascii,admin_name,capital
0,Casablanca,Casablanca,Casablanca-Settat,admin
1,Fès,Fes,Fès-Meknès,admin
2,Marrakech,Marrakech,Marrakech-Safi,admin
3,Tangier,Tangier,Tanger-Tétouan-Al Hoceïma,admin
4,Sale,Sale,Rabat-Salé-Kénitra,NaN


In [19]:
region_city['admin_name'].unique()

array(['Casablanca-Settat', 'Fès-Meknès', 'Marrakech-Safi',
       'Tanger-Tétouan-Al Hoceïma', 'Rabat-Salé-Kénitra', 'Oriental',
       'Souss-Massa', 'Béni Mellal-Khénifra', 'Laâyoune-Sakia El Hamra',
       'Guelmim-Oued Noun', 'Dakhla-Oued Ed-Dahab', 'Drâa-Tafilalet'],
      dtype=object)

In [20]:
region_city['city_ascii']=region_city['city_ascii'].str.lower()
region_city['admin_name']=region_city['admin_name'].str.lower()

In [21]:
region_city.isna().sum()

city            0
city_ascii      0
admin_name      0
capital       576
dtype: int64

In [22]:
region_city=region_city.drop(columns=['capital'])

In [23]:
region_city=region_city.rename(columns={'city_ascii': 'CITY_NAME'})

In [24]:
from unidecode import unidecode
region_city['admin_name'] = region_city['admin_name'].apply(lambda x: unidecode(x))

In [25]:
region_city['CITY_NAME'] = region_city['CITY_NAME'].str.replace("'", "", regex=False)

In [26]:
region_city.drop_duplicates()

,city,CITY_NAME,admin_name
0,Casablanca,casablanca,casablanca-settat
1,Fès,fes,fes-meknes
2,Marrakech,marrakech,marrakech-safi
3,Tangier,tangier,tanger-tetouan-al hoceima
4,Sale,sale,rabat-sale-kenitra
...,...,...,...
582,Lambarkiyine,lambarkiyine,casablanca-settat
583,Bir Anzarane,bir anzarane,dakhla-oued ed-dahab
584,Tifariti,tifariti,laayoune-sakia el hamra
585,Uad Damran,uad damran,laayoune-sakia el hamra


CLEANING HOLIDAY DATA

In [27]:
holidays.head()

,DAY_OFF,DAY_OFF_DESCRIPTION
0,01/01/2009,jour de l'an
1,11/01/2009,Manifeste de l'independance
2,10/03/2009,Aid Al Maoulid
3,11/03/2009,Aid Al Maoulid
4,28/03/2009,Migrations du DB2 et du CICS


In [28]:
holidays['DT'] = pd.to_datetime(holidays['DAY_OFF'], dayfirst=True, errors='coerce')
holidays=holidays.drop(columns=['DAY_OFF'])

In [29]:
holidays['DAY_OFF_DESCRIPTION']=holidays['DAY_OFF_DESCRIPTION'].str.lower()

In [30]:
holidays.isna().sum()

DAY_OFF_DESCRIPTION    0
DT                     0
dtype: int64

In [31]:
holidays

,DAY_OFF_DESCRIPTION,DT
0,jour de l'an,2009-01-01
1,manifeste de l'independance,2009-01-11
2,aid al maoulid,2009-03-10
3,aid al maoulid,2009-03-11
4,migrations du db2 et du cics,2009-03-28
...,...,...
256,marche verte,2024-11-06
257,fete de l'independance,2024-11-18
258,jour de l'an,2025-01-01
259,manifeste de l'indépendance,2025-01-11


MERGING DATASETS

In [32]:
first_merge= pd.merge(atm_cons,atm_city, on='TERMINAL', how='left')



In [33]:
first_merge

,TERMINAL,DT,AMOUNT,BRANCH_NAME,CITY_CODE,CITY_NAME
0,10000,2009-01-02,17200,zerktouni,780,casablanca
1,20000,2009-01-02,163300,zerktouni,780,casablanca
2,30000,2009-01-02,306900,centre d'affaire trabless,810,rabat
3,40000,2009-01-02,231400,mohammed v,810,rabat
4,50000,2009-01-02,241000,maamora,330,kenitra
...,...,...,...,...,...,...
8847237,25440000,2025-01-21,104600,ryad,810,rabat
8847238,25460000,2025-01-21,35000,zone franche,640,tanger
8847239,25470000,2025-01-21,67800,sebou,810,rabat
8847240,25500000,2025-01-21,8700,beni m'zala,726,fnidek


In [34]:
first_merge['CITY_NAME'] = first_merge['CITY_NAME'].str.strip().str.lower()
region_city['CITY_NAME'] = region_city['CITY_NAME'].str.strip().str.lower()


In [35]:
second_merge=pd.merge(first_merge,region_city, on='CITY_NAME', how='left')

In [36]:
second_merge.head()

,TERMINAL,DT,AMOUNT,BRANCH_NAME,CITY_CODE,CITY_NAME,city,admin_name
0,10000,2009-01-02,17200,zerktouni,780,casablanca,Casablanca,casablanca-settat
1,20000,2009-01-02,163300,zerktouni,780,casablanca,Casablanca,casablanca-settat
2,30000,2009-01-02,306900,centre d'affaire trabless,810,rabat,Rabat,rabat-sale-kenitra
3,40000,2009-01-02,231400,mohammed v,810,rabat,Rabat,rabat-sale-kenitra
4,50000,2009-01-02,241000,maamora,330,kenitra,Kenitra,rabat-sale-kenitra


In [37]:
common_cities = set(region_city['CITY_NAME']).intersection(set(first_merge['CITY_NAME']))
print(f"Number of common cities: {len(common_cities)}")
print("Common Cities:", common_cities)


Number of common cities: 121
Common Cities: {'ait melloul', 'biougra', 'saka', 'ketama', 'martil', 'aghbala', 'oulad said', 'chichaoua', 'midar', 'sale', 'dar ould zidouh', 'azrou', 'asilah', 'taourirt', 'zagora', 'azemmour', 'khenifra', 'oued amlil', 'farkhana', 'tetouan', 'al hoceima', 'oulmes', 'hassi berkane', 'aourir', 'ain cheggag', 'bouknadel', 'sidi bennour', 'sidi yahia el gharb', 'ait ourir', 'taineste', 'goulmima', 'rich', 'bouznika', 'fes', 'figuig', 'targuist', 'marrakech', 'ahfir', 'tissa', 'bhalil', 'guercif', 'ifrane', 'el ksiba', 'ain el aouda', 'saidia', 'nador', 'tineghir', 'el hajeb', 'oued zem', 'bni drar', 'kenitra', 'mrirt', 'driouch', 'errachidia', 'safi', 'tamri', 'ain aicha', 'el aioun', 'bni bouayach', 'imzouren', 'tit mellil', 'skoura', 'tata', 'ben ahmed', 'bni boufrah', 'berkane', 'tamanar', 'alnif', 'rabat', 'sidi ifni', 'boujad', 'el gara', 'sidi rahhal', 'madagh', 'bouskoura', 'ksar el kebir', 'taounate', 'moulay driss zerhoun', 'selouane', 'settat', 't

In [38]:
unique_cities_first_merge = set(first_merge['CITY_NAME'])
unique_cities_region_city = set(region_city['CITY_NAME'])
cities_only_in_first_merge = unique_cities_first_merge - unique_cities_region_city
print(f"Number of cities in first_merge but not in region_city: {len(cities_only_in_first_merge)}")
print("Cities only in first_merge:", cities_only_in_first_merge)


Number of cities in first_merge but not in region_city: 151
Cities only in first_merge: {'midelt', 'tan tan', 'mly bousselham', 'moulay abdellah', 'oulad abbou', 'iaazanan', 'bohouda', 'rissani', 'jorf el melha', 'chemaia', 'gzoula', 'ain taoujdate', 'taroudant', 'boudnib', 'oulad said el ouad', 'beni ansar', 'oualidiya', 'outat el haj', 'dakhla', 'fahs', 'bab berret', 'harhoura', 'had ait belfaa', 'bni ammart', 'mediek', 'dar el gueddari', 'nouaceur', 'tazaghine', 'azlaf', 'ait aatab', 'inchaden', 'sidi allal bahraoui', 'al aaouamra', 'chefchaoun', 'es semara', 'sidi kacem', 'mechra bel ksiri', 'oulad aamrane', 'tnine rharbia', 'el jadida', 'mhaya', 'el kelaa des sraghna', 'tleta akhssass', 'souk el arbaa el rharb', 'bab taza', 'ksibia dar bel amri', 'rhiate', 'ain bni mathar', 'zegangane', 'sebt oulad nemma', 'drargua', 'tarfaya', 'bni bou ifrour', 'taliouine', 'bouarfa', 'fnidek', 'taghzirt', 'bouizakarne', 'laayoune place', "oulad m'barek", 'ouazzane', 'beni tajjit', 'tahannaoute',

In [39]:
second_merge.isna().sum()

TERMINAL             0
DT                   0
AMOUNT               0
BRANCH_NAME          0
CITY_CODE            0
CITY_NAME            0
city           2595176
admin_name     2595176
dtype: int64

In [40]:
second_merge.head()

,TERMINAL,DT,AMOUNT,BRANCH_NAME,CITY_CODE,CITY_NAME,city,admin_name
0,10000,2009-01-02,17200,zerktouni,780,casablanca,Casablanca,casablanca-settat
1,20000,2009-01-02,163300,zerktouni,780,casablanca,Casablanca,casablanca-settat
2,30000,2009-01-02,306900,centre d'affaire trabless,810,rabat,Rabat,rabat-sale-kenitra
3,40000,2009-01-02,231400,mohammed v,810,rabat,Rabat,rabat-sale-kenitra
4,50000,2009-01-02,241000,maamora,330,kenitra,Kenitra,rabat-sale-kenitra


In [41]:
dict_cities = {
    "moulay abdellah": "Casablanca-Settat", "bab berret": "Casablanca-Settat", "bab taza": "Casablanca-Settat", 
    "sidi smail": "Casablanca-Settat", "tnine rharbia": "Casablanca-Settat", "debdou": "Oriental", 
    "ain defali": "Rabat-Sale-Kenitra", "ouarzazate": "Draa-Tafilalet", "gzoula": "Draa-Tafilalet", 
    "rissani": "Draa-Tafilalet", "imin tanoute": "Marrakech-Safi", "monte aruit / tiztoutine": "Oriental", 
    "taliouine": "Souss-Massa", "jorf al melha": "Rabat-Sale-Kenitra", "khnichet": "Rabat-Sale-Kenitra", 
    "sidi bettache": "Casablanca-Settat", "tafraoute": "Souss-Massa", "chtouka": "Souss-Massa", 
    "oulad m'barek": "Beni Mellal-Khenifra", "ouazzane": "Tanger-Tetouan-Al Hoceima", "chemaia": "Marrakech-Safi", 
    "zaida": "Draa-Tafilalet", "tleta akhssass": "Marrakech-Safi", "chefchaoun": "Tanger-Tetouan-Al Hoceima", 
    "mediek": "Souss-Massa", "massa": "Souss-Massa", "el kelaa des sraghna": "Marrakech-Safi", 
    "tan tan plage": "Guelmim-Oued Noun", "bohouda": "Draa-Tafilalet", "tafarsit": "Oriental", 
    "moulay yacoub": "Fes-Meknes", "sidi allal bahraoui": "Rabat-Sale-Kenitra", "aoulouz": "Souss-Massa", 
    "lqliaa": "Souss-Massa", "ras el ma": "Oriental", "bni yakhlaf": "Casablanca-Settat", "tahala": "Casablanca-Settat", 
    "el kelaa m gouna": "Draa-Tafilalet", "ben tib": "Casablanca-Settat", "sidi kacem": "Rabat-Sale-Kenitra", 
    "ouled frej": "Casablanca-Settat", "jorf": "Draa-Tafilalet", "bouarfa": "Oriental", "boudnib": "Draa-Tafilalet", 
    "ain bni mathar": "Oriental", "kariat arekmane": "Oriental", "oualidiya": "Casablanca-Settat", 
    "outat el haj": "Fes-Meknes", "had soualem": "Casablanca-Settat", "ksibia dar bel amri": "Rabat-Sale-Kenitra", 
    "midelt": "Draa-Tafilalet", "mechra bel ksiri": "Rabat-Sale-Kenitra", "tahar souk": "Fes-Meknes", 
    "sebt guerdane": "Rabat-Sale-Kenitra", "jemaa shim": "Marrakech-Safi", "jorf el melha": "Rabat-Sale-Kenitra", 
    "demnate": "Beni Mellal-Khenifra", "ajdir": "Tanger-Tetouan-Al Hoceima", "fkih ben salah": "Beni Mellal-Khenifra", 
    "zhiliga": "Souss-Massa", "tizi ouzli": "Souss-Massa", "oudaya": "Rabat-Sale-Kenitra", "bir gandouz": "Dakhla-Oued Ed-Dahab", 
    "rommani": "Rabat-Sale-Kenitra", "sebaa ayoune": "Souss-Massa", "boumalne dades": "Draa-Tafilalet", 
    "afourer": "Beni Mellal-Khenifra", "oulad said el ouad": "Beni Mellal-Khenifra", "tamallelte": "Souss-Massa", 
    "kassita": "Oriental", "el borouj": "Casablanca-Settat", "tlat loulad": "Casablanca-Settat", 
    "iaazanan": "Souss-Massa", "tamesna": "Rabat-Sale-Kenitra", "tanger": "Tanger-Tetouan-Al Hoceima", 
    "agdez": "Draa-Tafilalet", "sidi bouafif": "Tanger-Tetouan-Al Hoceima", "boujniba": "Beni Mellal-Khenifra", 
    "tazzarine": "Draa-Tafilalet", "tan tan": "Guelmim-Oued Noun", "bradia": "Casablanca-Settat", 
    "khemis zemamra": "Casablanca-Settat", "erfoud": "Draa-Tafilalet", "dar el gueddari": "Rabat-Sale-Kenitra", 
    "oulad berrehil": "Souss-Massa", "inchaden": "Souss-Massa", "zegangane": "Oriental", "oulad abbou": "Souss-Massa", 
    "berrechid": "Casablanca-Settat", "beni tajjit": "Souss-Massa", "mhaya": "Casablanca-Settat", 
    "el jadida": "Casablanca-Settat", "laayoune place": "Guelmim-Oued Noun", "mly bousselham": "Rabat-Sale-Kenitra", 
    "khemisset": "Rabat-Sale-Kenitra", "dar bouazza": "Casablanca-Settat", "had bni chiker": "Rabat-Sale-Kenitra", 
    "irherm": "Souss-Massa", "zaouia cheikh": "Tanger-Tetouan-Al Hoceima", "boulemane": "Oriental", 
    "had ait belfaa": "Tanger-Tetouan-Al Hoceima", "taznakht": "Draa-Tafilalet", "al aaouamra": "Rabat-Sale-Kenitra", 
    "rhiate": "Casablanca-Settat", "oulad aamrane": "Rabat-Sale-Kenitra", "khouribga": "Beni Mellal-Khenifra", 
    "es semara": "Dakhla-Oued Ed-Dahab", "dakhla": "Dakhla-Oued Ed-Dahab", "tahannaoute": "Marrakech-Safi", 
    "tazaghine": "Tanger-Tetouan-Al Hoceima", "tarfaya": "Guelmim-Oued Noun", "bni hafida": "Marrakech-Safi", 
    "bni ammart": "Tanger-Tetouan-Al Hoceima", "ain taoujdate": "Rabat-Sale-Kenitra", "beni ansar": "Tanger-Tetouan-Al Hoceima", 
    "dcheira": "Souss-Massa", "ben slimane": "Casablanca-Settat", "fnidek": "Tanger-Tetouan-Al Hoceima", 
    "ait ishak": "Souss-Massa", "souk el arbaa el rharb": "Casablanca-Settat", "ait aatab": "Souss-Massa", 
    "guelmin": "Souss-Massa", "kasba tadla": "Beni Mellal-Khenifra", "boufekrane": "Fes-Meknes", "fahs": "Fes-Meknes", 
    "nouaceur": "Casablanca-Settat", "azilal": "Beni Mellal-Khenifra", "harhoura": "Casablanca-Settat", 
    "boujdour": "Dakhla-Oued Ed-Dahab", "tinejdad": "Draa-Tafilalet", "sebt oulad nemma": "Beni Mellal-Khenifra", 
    "beni ahmed": "Marrakech-Safi", "aknoul": "Fes-Meknes", "bni bou ifrour": "Fes-Meknes", "bouizakarne": "Souss-Massa", 
    "smimou": "Marrakech-Safi", "oujda": "Oriental", "had kourt": "Rabat-Sale-Kenitra", "oulad yaiche": "Fes-Meknes", 
    "taghzirt": "Rabat-Sale-Kenitra", "ribat el kheir": "Marrakech-Safi", "tleta jbel el habib": "Souss-Massa", 
    "sidi hajjaj": "Souss-Massa", "drargua": "Marrakech-Safi", "attaouia ech chaibiya": "Rabat-Sale-Kenitra", 
    "ras el ain": "Marrakech-Safi", "aoufouss": "Souss-Massa", "dar kebdani": "Beni Mellal-Khenifra", 
    "ouaouizarht": "Souss-Massa", "azlaf": "Dakhla-Oued Ed-Dahab", "taroudant": "Souss-Massa"
}

second_merge['admin_name'] = second_merge.apply(
    lambda row: dict_cities.get(row['CITY_NAME'].lower(), row['admin_name']) 
    if pd.isna(row['admin_name']) and row['CITY_NAME'].lower() in dict_cities else row['admin_name'],
    axis=1
)



In [42]:
merged_data = second_merge.merge(holidays, on='DT', how='left')

In [43]:
merged_data.head()

,TERMINAL,DT,AMOUNT,BRANCH_NAME,CITY_CODE,CITY_NAME,city,admin_name,DAY_OFF_DESCRIPTION
0,10000,2009-01-02,17200,zerktouni,780,casablanca,Casablanca,casablanca-settat,NaN
1,20000,2009-01-02,163300,zerktouni,780,casablanca,Casablanca,casablanca-settat,NaN
2,30000,2009-01-02,306900,centre d'affaire trabless,810,rabat,Rabat,rabat-sale-kenitra,NaN
3,40000,2009-01-02,231400,mohammed v,810,rabat,Rabat,rabat-sale-kenitra,NaN
4,50000,2009-01-02,241000,maamora,330,kenitra,Kenitra,rabat-sale-kenitra,NaN


In [44]:
merged_data.rename(columns={'DT': 'date'}, inplace=True)
merged_data.rename(columns={'AMOUNT': 'amount'}, inplace=True)
merged_data.rename(columns={'CITY_NAME': 'city'}, inplace=True)
merged_data.rename(columns={'admin_name': 'region'}, inplace=True)
merged_data.rename(columns={'DAY_OFF_DESCRIPTION': 'day_off_discription'}, inplace=True)
merged_data.rename(columns={'TERMINAL': 'atm_id'}, inplace=True)

In [45]:
merged_data['atm_id'] = merged_data['atm_id'].astype(str).str.replace(r'0000$', '', regex=True).astype(int)


In [46]:
merged_data=merged_data.drop_duplicates(subset=['atm_id','date'])

In [47]:
merged_data.head()

,atm_id,date,amount,BRANCH_NAME,CITY_CODE,city,city,region,day_off_discription
0,1,2009-01-02,17200,zerktouni,780,casablanca,Casablanca,casablanca-settat,NaN
1,2,2009-01-02,163300,zerktouni,780,casablanca,Casablanca,casablanca-settat,NaN
2,3,2009-01-02,306900,centre d'affaire trabless,810,rabat,Rabat,rabat-sale-kenitra,NaN
3,4,2009-01-02,231400,mohammed v,810,rabat,Rabat,rabat-sale-kenitra,NaN
4,5,2009-01-02,241000,maamora,330,kenitra,Kenitra,rabat-sale-kenitra,NaN


In [48]:
merged_data.to_csv('../data/cleaned/merged_atm_data.csv', index=False, chunksize=100000)